In [1]:
import pandas as pd
import numpy as np
import altair as alt
from covid import dataimport

base_url = "https://raw.githubusercontent.com/morberg/covid-notebook/master/data/"
combined_url = base_url + "combined.json"
scb_url = base_url + "scb.json"

In [2]:
data_scb = dataimport.get_scb_county_data()
latest_data_point = data_scb.date.max()

In [3]:
today = pd.datetime.today()

# Excess mortality in Sweden 2020

Charts that show excess mortality in 2020 compared to 2018-2019. Based on data [published weekly by Statistiska Centralbyrån](https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/). Due to reporting lag the data for the last two weeks will change.

## Total weekly deaths 

Numbers for Covid-19 deaths from FHM marked in orange.

In [4]:
base = alt.Chart('data/combined.json', height=400, width=750, title={
        "text": 'Weekly Deaths in Sweden 2018-2020',
        "subtitle": f'Data published {latest_data_point.date().isoformat()} by SCB'}).encode(
    x=alt.X('monthdate(date):T', axis=alt.Axis(format="%b"), title='Month')
)

history = base.mark_line(size=3, opacity=1).transform_filter(
    'year(datum.date) < 2020'
).transform_aggregate(
    sum_deaths='sum(deaths):Q',
    groupby=['date']
).encode(
    y='mean(sum_deaths):Q',
    color=alt.Color('year(date):N', title='', scale=alt.Scale(range=['#b9b9b9', '#898989']))
)

covid_base = base.encode(
    y=alt.Y('deaths:Q', title='Weekly Deaths'),
    color=alt.Color('death_cause:N', title='2020', sort='descending')
).transform_filter(
    'year(datum.date) == 2020'
)

# Should really use latest data point here instead of now().
# Don't know how to get that into the vega lite expression.
covid = covid_base.mark_bar(size=14).transform_filter(
    'dayofyear(datum.date) <= (dayofyear(now()) - 14)'
)

covid_uncertain = covid_base.mark_bar(size=14,opacity=0.3).encode(
    color=alt.Color('death_cause:N', title='2020', sort='descending', legend=None)

)

(covid + covid_uncertain + history).resolve_scale(color='independent')

alt.LayerChart(...)

## Total deaths with filter on county level

Click on a county marker in the legend to the right to include/exclude it in the total numbers.

In [5]:
last_week_to_show = latest_data_point.week
filter = f'year(datum.date) == 2020 & week(datum.date) < {last_week_to_show}'

base = alt.Chart(scb_url, height=400, width=700, title={
        "text": 'Weekly Deaths in Sweden 2018-2020',
        "subtitle": f'Data published {latest_data_point.date().isoformat()} by SCB'}).encode(
    x=alt.X('monthdate(date):T', axis=alt.Axis(format="%b"), title=None),
    y=alt.Y('sum(deaths):Q', title='Weekly Deaths'),
)

history = base.mark_line(opacity=0.4, size=2).encode(color=alt.Color('year(date):N', title='Year')).transform_filter(
    "year(datum.date) < 2020"
)

current = base.encode().mark_line(size=5, opacity=1).transform_filter(
    filter
).encode(
    y='sum(deaths):Q',
    color=alt.Color('year(date):N', title=None, scale=alt.Scale(range=['#e45757']))
)

selection = alt.selection_multi(toggle='true', encodings=['y'])

legend = alt.Chart(scb_url, height=320, width=60).mark_square(size=150).encode(
    y=alt.Y('county:N', axis=alt.Axis(orient='right'), title=None),
    x=alt.X('sum(deaths):Q', title='Total Deaths 2020'),
    color=alt.condition(selection, alt.value('#e45757'), alt.value('lightgrey')),
    tooltip=[alt.Tooltip('sum(deaths):Q', title='Total Deaths 2020'), 'county:N'],
).add_selection(
    selection
).transform_filter(
    filter
).interactive()

(history + current).transform_filter(selection).resolve_scale(color='independent') | legend

alt.HConcatChart(...)

## Cumulative deaths 2018-2020 by County

An illustration of excess deaths in 2020 per county compared to 2018 and 2019.

In [6]:
alt.Chart(scb_url, height=160, width=160).mark_line().encode(
    x=alt.X('monthdate(date):T', title='Date', axis=alt.Axis(format="%b")),
    y=alt.Y('cumulative_deaths:Q', title=''),
    color=alt.Color('year(date):N', title='Year', scale=alt.Scale(range=['#d9d9d9', '#b9b9b9', '#e6550d'])),
    tooltip=['date:T', alt.Tooltip('cumulative_deaths:Q', title='Deaths')]
).facet(
    facet=alt.Facet("county:N", title="Cumulative Deaths 2018-2020 by County"),
    columns=4,
).resolve_scale(y='independent')

alt.FacetChart(...)